In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
import re
import string
import nltk
from nltk.corpus import stopwords
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from wordcloud import WordCloud,STOPWORDS
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer
import nltk, os, re, string, collections
import matplotlib
from  imageio import imread
from nltk.util import ngrams
import itertools
import spacy
from sklearn.preprocessing import MultiLabelBinarizer, Binarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from collections import Counter
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import lightgbm as lgb
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
import gc
from mlxtend.classifier import StackingClassifier
from bayes_opt import BayesianOptimization
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
import pickle
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [45]:
train = pd.read_csv('original_data/train.csv')
test  = pd.read_csv('original_data/test.csv')

# 1) LIMPIEZA DE TEXTO

## 1) a) Funciones

In [46]:
def getStopwordsList():
    fileNamesList = ["texts/99webtools.txt", "texts/atire_ncbi.txt", "texts/atire_puurula.txt", "texts/azure.txt", 
                 "texts/bbalet.txt", 
                 "texts/bow_short.txt", "texts/choi_2000naacl.txt", "texts/cook1988_function_words.txt", 
                 "texts/corenlp_acronym.txt", 
                 "texts/corenlp_hardcoded.txt", "texts/corenlp_stopwords.txt", "texts/datasciencedojo.txt", 
                 "texts/deeplearning4j.txt", 
                 "texts/dkpro.txt", "texts/mongodb.txt", "texts/galago_inquery.txt", "texts/gate_keyphrase.txt", 
                 "texts/gensim.txt", 
                 "texts/glasgow_stop_words.txt", "texts/indri.txt", "texts/kevinbouge.txt", "texts/lexisnexis.txt",
                 "texts/lingpipe.txt", 
                 "texts/mallet.txt", "texts/mysql_innodb.txt", "texts/mysql_myisam.txt", "texts/galago_rmstop.txt", 
                 "texts/atire_ncbi.txt", 
                 "texts/galago_rmstop.txt", "texts/nltk.txt", "texts/okapiframework.txt", "texts/okapi_cacm_expanded.txt", 
                 "texts/onix.txt", 
                 "texts/ovid.txt", "texts/postgresql.txt", "texts/pubmed.txt", "texts/quanteda.txt", "texts/r_tm.txt", 
                 "texts/ranksnl_large.txt", 
                 "texts/reuters_wos.txt", "texts/rouge_155.txt", "texts/scikitlearn.txt", "texts/smart.txt", 
                 "texts/snowball_expanded.txt", 
                 "texts/spacy.txt", "texts/spark_mllib.txt", "texts/sphinx_mirasvit.txt", "texts/t101_minimal.txt", 
                 "texts/taporware.txt", 
                 "texts/terrier.txt", "texts/tonybsk_1.txt", "texts/tonybsk_6.txt", "texts/voyant_taporware.txt", 
                 "texts/weka.txt", 
                 "texts/xapian.txt", "texts/xpo6.txt", "texts/zettair.txt"]
    stopwordsList = []
    for fileName in fileNamesList:
        file = open(fileName, "r", encoding="utf8")
        for line in file:
            stripped_line = line. strip()
            line_list = stripped_line
            if line_list not in stopwordsList:
                stopwordsList.append(line_list)
        file.close()
    return stopwordsList

stopwordsList = getStopwordsList()

print(len(stopwordsList))

2253


In [47]:
def clean_text_strict(text):
    text = text.lower()
    text = text.replace('\d+', '')
    invalid_chars = ['#','|','@','!','?','-','_','[',']','%','&',':','.',',',"''",'/','https','(','//t',')','http',
                 ';','\'']
    for char in invalid_chars:
        if char in text:
            text = text.replace(char,' ')
    #removes url and tags
    text = re.sub(r"(?:\@|https?\://)\S+", " ",  text)
    return text

In [48]:
# FUNCIONES AUXILIARES

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def prettify(word):
    word = re.sub(r'[^A-Za-z]', " ",  word)
    word = re.sub(r'\b\w\b',' ', word)
    word = re.sub(r'\s+', ' ', word)
    return word

def cleanStopwords(tweet):
    tweet = tweet.split(' ')
    cleanTweet=""
    for word in tweet:
        if word not in stopwordsList:
            cleanTweet = cleanTweet + " " + word
    return cleanTweet

def clean_text_non_strict(text): 
    tw = " "
    words = text.split(' ')
    for word in words:
        word = ''.join(filter(lambda x: x in set(string.printable), word))
        word = word.replace("\n"," ")
        word = word.replace('û',"")
        word = word.replace('Û',"")
        #word = word.replace('_','')
        #word = word.replace("\"",'')
        #word = word.strip('.')
        #word = word.strip(',')
        #word = word.strip(':')
        tw += word + " "
    return tw

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])
def lemmatize_text_v2(text):
    if text == '' or pd.isnull(text):
        return text
    else:
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])

In [ ]:
#FUNCIONES AUXILIARES: Limpieza de tweet

# Función que saca los links

def quitar_link_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if (("http" not in w) and ("https" not in w)):
            res.append(w)
    return (" ").join(res)

# Función que saca las menciones

def quitar_mencion_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if("@" not in w):
            res.append(w)
    return (" ").join(res)

def quitar_simbolo(tweet, simbolo):
    text = ""
    for c in tweet:
        if(c!=simbolo):
            text+=c
    return text

def agregar_espacio_ente_comas(tweet):
    
    tweer_sin_coma = (" ").join(tweet.split(","))

    return (" ").join(tweer_sin_coma.split("."))

def capitalize_each_word(string):
    
    text = ""
    for w in string.split(" "):
        if(len(w)>3):
            text += w.capitalize()+" "
    return text

def limpiar_tweet_location(tweet):
    res = tweet
    func = [quitar_mencion_twitter, quitar_link_twitter, agregar_espacio_ente_comas, lambda x: quitar_simbolo(x, "#"), capitalize_each_word]
    for f in func:
        res = f(res)
    return res

def to_string(a_list, delimeter):
    res = ""
    
    for e in sorted(filter(None, a_list)):
        res += str(e)+delimeter
    
    if(len(res)==0): return None
    return res[: -len(delimeter)]

In [ ]:
def addKeywordToText(x):
    y=pd.isnull(x.keyword)
    if (y == True):
        return x['text']
    else:
        return (x['keyword'] + ' ' + x['text'])

In [ ]:
def addLocationToText(x):
    if (x.location == ' '):
        return x['text_w_location']
    else:
        return (x['location'] + ' ' + x['text_w_location'])

## 1) b) Text + Keyword [baja un toque el score]

In [ ]:
'''train["keyword"] = train["keyword"].str.replace('%20',' ')
train['text_w_keyword'] = train.apply(lambda x: addKeywordToText(x), axis=1)

test["keyword"] = test["keyword"].str.replace('%20',' ')
test['text_w_keyword'] = test.apply(lambda x: addKeywordToText(x), axis=1)

train.text_w_keyword = train["text_w_keyword"].apply(lambda x: clean_text_strict(x))
test.text_w_keyword  = test["text_w_keyword"].apply(lambda x: clean_text_strict(x))

train.text_w_keyword = train.text_w_keyword.apply(prettify)
test.text_w_keyword  = test.text_w_keyword.apply(prettify)

train.text_w_keyword = train.text_w_keyword.apply(lemmatize_text_v2)
test.text_w_keyword  = test.text_w_keyword.apply(lemmatize_text_v2)

train.text_w_keyword = train.text_w_keyword.apply(cleanStopwords)
test.text_w_keyword  = test.text_w_keyword.apply(cleanStopwords)'''

## 1) c) Text + Location [baja score :(]

In [ ]:
'''train.location = train.location.fillna('')
train.location = train.location.apply(lambda x:limpiar_tweet_location(x))
train['text_w_location'] = train.apply(lambda x: addLocationToText(x), axis=1)

test.location = test.location.fillna('')
test.location = test.location.apply(lambda x:limpiar_tweet_location(x))
test['text_w_location'] = test.apply(lambda x: addLocationToText(x), axis=1)

train.text_w_location = train["text_w_location"].apply(lambda x: clean_text_strict(x))
test.text_w_location  = test["text_w_location"].apply(lambda x: clean_text_strict(x))

train.text_w_location = train.text_w_location.apply(prettify)
test.text_w_location  = test.text_w_location.apply(prettify)

train.text_w_location = train.text_w_location.apply(lemmatize_text_v2)
test.text_w_location  = test.text_w_location.apply(lemmatize_text_v2)

train.text_w_location = train.text_w_location.apply(cleanStopwords)
test.text_w_location  = test.text_w_location.apply(cleanStopwords)'''

## 1) d)  Sólo Texto [mejor hasta ahora]

In [ ]:
#train.text = train["text"].apply(lambda x: quitar_link_twitter(x)) #Por algun motivo esto baja un poquito el score
#test.text  = test["text"].apply(lambda x: quitar_link_twitter(x))

#train.text = train["text"].apply(lambda x: quitar_mencion_twitter(x)) #Por algun motivo esto baja un poquito el score
#test.text  = test["text"].apply(lambda x: quitar_mencion_twitter(x))

train.text = train["text"].apply(lambda x: clean_text_non_strict(x))
test.text  = test["text"].apply(lambda x: clean_text_non_strict(x))

train.text = train["text"].apply(lambda x: clean_text_strict(x))
test.text  = test["text"].apply(lambda x: clean_text_strict(x))

In [ ]:
train.text = train.text.apply(prettify)
test.text  = test.text.apply(prettify)

In [49]:
# a veces baja score
#train.text = train.text.apply(lemmatize_text_v2)
#test.text  = test.text.apply(lemmatize_text_v2)

In [75]:
train.text

0        -PRON- deed reason earthquake allah forgive -...
1                           forest fire ronge sask canada
2        resident shelter notify officer evacuation sh...
3           people receive wildfire evacuation california
4                   photo ruby alaska smoke wildfire pour
                              ...                        
7608     giant crane hold bridge collapse nearby stfmb...
7609       aria ahrary thetawniest control wild fire c...
7610                        utc volcano hawaii zdtoyd ebj
7611     police bike collide portugal bike rider suffe...
7612       raze northern california wildfire abc ymy rskq
Name: text, Length: 7613, dtype: object

In [51]:
train.text = train.text.apply(cleanStopwords)
test.text  = test.text.apply(cleanStopwords)

In [52]:
train.text

0       our deeds are the reason of this earthquake ma...
1                   forest fire near la ronge sask canada
2       all residents asked to shelter in place are be...
3        people receive wildfires evacuation orders in...
4       just got sent this photo from ruby alaska as s...
                              ...                        
7608    two giant cranes holding bridge collapse into ...
7609     aria ahrary thetawniest the out of control wi...
7610               utc km of volcano hawaii co zdtoyd ebj
7611    police investigating after an bike collided wi...
7612    the latest more homes razed by northern califo...
Name: text, Length: 7613, dtype: object

# 2) FEATURES NUMERICOS

## 2) a) Preparation of the TRAIN and TEST set

In [ ]:
# TEST SET - ONLY NUMERICAL
test_keywords_numerical = pd.read_csv('csv_lio/test/keywords_numerical_features_test - Copy.csv')
test_keywords_numerical = test_keywords_numerical.drop(columns = 'Unnamed: 0')
test_text_numerical = pd.read_csv('csv_lio/test/text_general_numerical_features_test - Copy.csv')
test_text_numerical = test_text_numerical.drop(columns = ['Unnamed: 0','#silabas'])
test_hashtag_numerical = pd.read_csv('csv_lio/test/features_hashtags_numerical.csv')
test_hashtag_numerical  =test_hashtag_numerical.drop(columns = 'Unnamed: 0')
test_links_numerical = pd.read_csv('csv_lio/test/features_links_numerical.csv')
test_links_numerical = test_links_numerical.drop(columns = ['id.1','target'])
test_new_use = test.drop(columns = ['keyword','location'])

In [ ]:
features_test = pd.merge(test_new_use,test_text_numerical, on = 'id', how = 'left')
features_test = pd.merge(features_test,test_keywords_numerical, on = 'id', how = 'left')
features_test = pd.merge(features_test,test_hashtag_numerical, on = 'id', how = 'left')
features_test = pd.merge(features_test,test_links_numerical, on = 'id', how = 'left')

In [ ]:
# TEST: Df with general numerical features
features_test.head()

In [ ]:
# TRAIN SET - ONLY NUMERICAL
train_keywords_numerical = pd.read_csv('csv_lio/train/keywords_numerical_features - Copy.csv')
train_keywords_numerical = train_keywords_numerical.drop(columns = ['Unnamed: 0','target'], axis = 1)
train_text_numerical = pd.read_csv('csv_lio/train/text_general_numerical_features_train - Copy.csv')
train_text_numerical = train_text_numerical.drop(columns = ['Unnamed: 0','target','#silabas'])
train_hashtag_numerical = pd.read_csv('csv_lio/train/features_hashtags_numerical.csv')
train_hashtag_numerical  =train_hashtag_numerical.drop(columns = ['Unnamed: 0','target'])
train_links_numerical = pd.read_csv('csv_lio/train/features_links_numerical.csv')
train_links_numerical = train_links_numerical.drop(columns = ['id.1','target'])
train_new_use = train.drop(columns = ['keyword','location'])

In [ ]:
features_train = pd.merge(train_new_use,train_text_numerical, on = 'id', how = 'left')
features_train = pd.merge(features_train,train_keywords_numerical, on = 'id', how = 'left')
features_train = pd.merge(features_train,train_hashtag_numerical, on = 'id', how = 'left')
features_train = pd.merge(features_train,train_links_numerical, on = 'id', how = 'left')

In [ ]:
# TRAIN: Df with general numerical features
features_train.head()

## 2) b) Transformer-creation

In [ ]:
# Text transformer: it selects a single column from the data frame to perform additional transformations on
from sklearn.base import BaseEstimator, TransformerMixin
class TextSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

# Value transformer:  it selects a single column from the data frame to perform additional transformations on
class NumberSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

## 2) d) Now, a PIPELINE for each feature in the df is created

In [ ]:
# For TEXT -this pipeline consists of "selecting" and then "bow-ing" a column.
bow_pipeline = Pipeline([
                ('selector', TextSelector(key='text')),
                ('bow',CountVectorizer(analyzer='word',binary=True,ngram_range=(1,2)))
            ])

In [ ]:
# For TEXT -  this pipeline consists of "selecting" and then "td_idf-ing" a column.
tfidf_pipeline = Pipeline([
                ('selector', TextSelector(key='text')),
                ('tfidf', TfidfVectorizer( ))
            ])

In [ ]:
# For TEXT -  this pipeline consists of "selecting" and then "td_idf-ing" a column.
tfidf2_pipeline = Pipeline([
                ('selector', TextSelector(key='text')),
                ('tfidf', TfidfVectorizer(use_idf=True,smooth_idf=True,ngram_range=(1,1),stop_words='english'))
            ])

In [ ]:
# For TEXT -  this pipeline consists of "selecting" and then "td_idf-ing" a column.
tfidf3_pipeline = Pipeline([
                ('selector', TextSelector(key='text')),
                ('tfidf', TfidfVectorizer(use_idf=True,smooth_idf=True,ngram_range=(1,2),stop_words='english'))
            ])

In [ ]:
# For TEXT -  this pipeline consists of "selecting" and then "td_idf-ing" a column.
tfidf4_pipeline = Pipeline([
                ('selector', TextSelector(key='text')),
                ('tfidf', TfidfVectorizer(use_idf=True,smooth_idf=True,ngram_range=(1,2)))
            ])

In [ ]:
def numerical_pipeline_creator(df_train):
    #df.columns.tolist()
    feature_names = df_train.columns.tolist()
    pipelines = []
    for feature_name in feature_names:
        current_pipeline =  Pipeline([
                            ('selector', NumberSelector(key=feature_name)),
                            #('standard', StandardScaler())
                        ])
        pipelines.append(current_pipeline)
        
    return pipelines

In [ ]:
# list_numerical_pipelines contains all pipelines for all numerical features. To see the correspondance between the array and the
# corresponding feature, get the column names of X_train_numerical df
train_numerical = features_train.iloc[:,3:31]
list_numerical_pipelines = numerical_pipeline_creator(train_numerical)

In [ ]:
feature_names = train_numerical.columns.tolist()
for feature_name in feature_names: 
    print(feature_name)

In [ ]:
# Pipelines according to feature name
cant_palabras= list_numerical_pipelines[0]
cant_palabras_unicas= list_numerical_pipelines[1]
cant_caracteres= list_numerical_pipelines[2]
cant_stopwords= list_numerical_pipelines[3]
cant_puntuacion= list_numerical_pipelines[4]
cant_capitalize= list_numerical_pipelines[5]
cant_mayusculas= list_numerical_pipelines[6]
promedio_len_word= list_numerical_pipelines[7]
cant_caracteres_especiales= list_numerical_pipelines[8]
cant_palabras_binned= list_numerical_pipelines[9]
cant_palabras_unicas_binned= list_numerical_pipelines[10]
cant_caracteres_binned= list_numerical_pipelines[11]
cant_stopwords_binned= list_numerical_pipelines[12]
cant_puntuacion_binned= list_numerical_pipelines[13]
cant_capitalize_binned= list_numerical_pipelines[14]
cant_mayusculas_binned= list_numerical_pipelines[15]
cant_silabas_binned= list_numerical_pipelines[16]
cant_caracteres_especiales_binned= list_numerical_pipelines[17]
text_contains_keyword= list_numerical_pipelines[18]
has_keyword= list_numerical_pipelines[19]
keywords_quantity= list_numerical_pipelines[20]
keywords_mean= list_numerical_pipelines[21]
keyword_is_hashtag= list_numerical_pipelines[22]
keyword_frequency= list_numerical_pipelines[23]
cant_hashtags= list_numerical_pipelines[24]
has_hashtag= list_numerical_pipelines[25]
links_cant= list_numerical_pipelines[26]
cant_failed_links= list_numerical_pipelines[27]

# 3) MODELOS

## 3) a) Use of FeatureUnion to put features together

In [ ]:
# ATTENTION: Each line is a new feature, so if you wanna add a new feature to the analysis, simply remove the hashtag and run it
# again.
# If new feature which were not created are to be added, create first and then add it to this group in a new line
feats = FeatureUnion([               #   ('cant_palabras', cant_palabras), 
                                     #   ('cant_palabras_unicas', cant_palabras_unicas),
                                     #   ('cant_caracteres', cant_caracteres),
                                     #   ('cant_stopwords', cant_stopwords), # parece que overfittea
                                     #   ('cant_puntuacion', cant_puntuacion),
                                     #   ('cant_capitalize', cant_capitalize),
                                     #   ('cant_mayusculas',cant_mayusculas), #parece que overfittea
                                         ('promedio_len_word',promedio_len_word),
                                     #   ('cant_caracteres_especiales',cant_caracteres_especiales),
                                     #   ('cant_palabras_binned',cant_palabras_binned),
                                     #   ('cant_palabras_unicas_binned',cant_palabras_unicas_binned),
                                     #   ('cant_caracteres_binned',cant_caracteres_binned),
                                     #   ('cant_stopwords_binned',cant_stopwords_binned),
                                     #   ('cant_puntuacion_binned',cant_puntuacion_binned),
                                     #   ('cant_capitalize_binned',cant_capitalize_binned),
                                     #   ('cant_mayusculas_binned',cant_mayusculas_binned),
                                     #   ('cant_silabas_binned',cant_silabas_binned),
                                     #   ('cant_caracteres_especiales_binned',cant_caracteres_especiales_binned),
                                     #   ('text_contains_keyword',text_contains_keyword),
                                     #   ('has_keyword',has_keyword),
                                     #   ('keywords_quantity',keywords_quantity),
                                     #   ('keywords_mean',keywords_mean),
                                     #   ('keyword_is_hashtag',keyword_is_hashtag),
                                     #   ('keyword_frequency',keyword_frequency),
                                     #   ('cant_hashtags',cant_hashtags),
                                     #   ('has_hashtag',has_hashtag),
                                     #   ('links_cant',links_cant),
                                     #   ('cant_failed_links',cant_failed_links),
                                     #   ('text_pipeline',text_pipeline)
                                     #   ('bow', bow_pipeline),
                                        ('bow2', bow2_pipeline),
                                        ('tfidf', tfidf_pipeline),
                                     #   ('tfidf2', tfidf2_pipeline),
                                     #   ('tfidf3', tfidf3_pipeline),
                                     #   ('tfidf4', tfidf4_pipeline),
                     ])

feats_lr = FeatureUnion([   ('cant_caracteres', cant_caracteres),
                            ('cant_capitalize', cant_capitalize),
                            ('promedio_len_word',promedio_len_word),
                            ('keyword_frequency',keyword_frequency),
                            ('bow2', bow2_pipeline)
                     ])

feats_nb = FeatureUnion([   ('cant_palabras', cant_palabras), 
                            ('cant_palabras_unicas', cant_palabras_unicas),
                            ('cant_caracteres', cant_caracteres),
                            ('cant_capitalize', cant_capitalize),
                            ('promedio_len_word',promedio_len_word),
                            ('cant_caracteres_especiales',cant_caracteres_especiales),
                            ('has_keyword',has_keyword),
                            ('keyword_frequency',keyword_frequency),
                            ('bow2', bow2_pipeline)
                     ])

feats_rf = FeatureUnion([   ('cant_palabras', cant_palabras), 
                            ('cant_caracteres', cant_caracteres),
                            ('cant_stopwords', cant_stopwords),
                            ('cant_capitalize', cant_capitalize),
                            ('promedio_len_word',promedio_len_word),
                            ('text_contains_keyword',text_contains_keyword),
                            ('keywords_quantity',keywords_quantity),
                            ('keywords_mean',keywords_mean),
                            ('keyword_frequency',keyword_frequency),
                            ('cant_hashtags',cant_hashtags),
                            ('links_cant',links_cant),
                            ('bow2', bow2_pipeline),
                            ('tfidf', tfidf_pipeline)
                     ])

feats_sgd_svc = FeatureUnion([  ('cant_palabras_unicas_binned',cant_palabras_unicas_binned),
                                ('cant_puntuacion_binned',cant_puntuacion_binned),
                                ('cant_caracteres_especiales_binned',cant_caracteres_especiales_binned),
                                ('text_contains_keyword',text_contains_keyword),
                                ('keyword_is_hashtag',keyword_is_hashtag),
                                ('keyword_frequency',keyword_frequency),
                                ('has_hashtag',has_hashtag),
                                ('cant_failed_links',cant_failed_links),
                                ('bow2', bow2_pipeline)
                     ])

feats_sgd_lr = FeatureUnion([   ('cant_palabras_unicas_binned',cant_palabras_unicas_binned),
                                ('cant_puntuacion_binned',cant_puntuacion_binned),
                                ('cant_caracteres_especiales_binned',cant_caracteres_especiales_binned),
                                ('text_contains_keyword',text_contains_keyword),
                                ('keyword_is_hashtag',keyword_is_hashtag),
                                ('keyword_frequency',keyword_frequency),
                                ('has_hashtag',has_hashtag),
                                ('cant_failed_links',cant_failed_links),
                                ('bow2', bow2_pipeline),
                                ('tfidf', tfidf_pipeline)
                     ])


feature_processing = Pipeline([('feats', feats)])
feature_processed = feature_processing.fit_transform(features_train)

## 3) b) Dani's junk

In [35]:
def evaluate(model, test_features, test_labels):
    
    predictions = model.predict(test_features)
    accuracy = round(accuracy_score(test_labels,predictions)*100)
    
    print("\nMODEL")
    
    print(model)
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    print()
    return accuracy

In [ ]:
# This funtion saves the value of the hyperparameters and the score of the validation train set 
def save_prediction_accuracy(modelo,x_train,y_train,x_test,y_test):
    clf = modelo
    clf.fit(x_train, y_train)
    predictions = clf.predict(x_test)
    with open("prediction_history.csv", "a") as myfile:
        #round(clf.score(X_train, y_train)*100)
        train_prediction = str((clf.score(X_train, y_train)*100))
        print('Score para x_train: '+ train_prediction)
        test_prediction = str((accuracy_score(y_test,predictions)*100))
        print('Score para x_test: '+ test_prediction)
        params = str(modelo)
        print('Hiperparametros: '+ str(modelo))
        myfile.write(params+','+test_prediction+","+train_prediction+","+str(datetime.datetime.now())+"\n")

In [ ]:
def save_prediction(model, test, name):
    predicted = model.predict(test)
    sample_submission.target = predicted
    sample_submission.to_csv('predictions/'+name+'.csv', index=None)
    return

In [ ]:
sample_submission = pd.read_csv('original_data/sample_submission.csv')

In [ ]:
y = features_train.target
X = features_train.drop(columns=['target','id'])

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(train_vec,y,test_size=0.2,random_state=2020)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

# 4) Estimación sobre modelos

In [58]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [59]:
def fit_and_predict(model,X_train,y_train,X_test,y_test):    
    clf = model
    clf.fit(X_train, y_train)
    predictions=clf.predict(X_test)
    confusion_matrix(y_test,predictions)
    conf = metrics.confusion_matrix(y_test, predictions)
    conf = conf.astype('float') / conf.sum(axis=1)[:, np.newaxis]
    print_cm(conf, ['true','false'])
    print('-'*50)
    print(classification_report(y_test,predictions))
    print('-'*50)
    print("{}" .format(model))
    print('-'*50)
    print('Accuracy of classifier on training set:{}%'.format(round(clf.score(X_train, y_train)*100)))
    print('-'*50)
    print('Accuracy of classifier on test set:{}%' .format(round(accuracy_score(y_test,predictions)*100)))
    print('*'*50)

In [ ]:
# This function applies different ESTIMATORS -with specific or default hyperparameters- to a set of features
def runClassifiers(X_train, y_train, X_test,y_test):
    classifiers = [LogisticRegression(),MultinomialNB(alpha=1),DecisionTreeClassifier(),RandomForestClassifier(),
                   SGDClassifier(loss='hinge', penalty='l2', alpha=0.001, random_state=42, max_iter=5, tol=None),
                   SGDClassifier(loss='log', penalty='l2', alpha=0.001, random_state=42, max_iter=5, tol=None),
                   KNeighborsClassifier(n_neighbors=3, weights = 'distance',p = 1)
                 ]

    classifier_names = ['Logistic Regression','MultinomialNB','Decision-Tree Classifier',
                       'Random-Forest Classifier','SGDC - svc - Classifier','SGDC - lr - Classifier','K-Neighbors Classifier']
    features = [feats_lr, feats_nb, feats, feats_rf, feats_sgd_svc, feats_sgd_lr, feats]
    
    i=-1
    for classifier in classifiers:
        i = i + 1
        clf = Pipeline([
            ('features', features[i]),
            ('actualClassifier', classifier )
        ])
        print(classifier_names[i])
        fit_and_predict(clf,X_train,y_train,X_test,y_test)
        #clf.fit(X_train, y_train)
        #docs_test = X_test
        #predicted = clf.predict(docs_test)
        #print(classifier_names[i],np.mean(predicted == y_test))


In [60]:
#probando de todo
runClassifiers(X_train,y_train,X_test,y_test)

## Estimación con Cross Validation

In [3]:
n=5
seed=2020
kf = KFold(n_splits=n, random_state=seed, shuffle=True)
 
def acc_cross_val(classifier, feats, X):
    # Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
    accuracy_model = []
    # Iterate over each train-test split
    for train_index, test_index in kf.split(X):
        # Split train-test
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Train the model
        clf = Pipeline([
            ('features', feats),
            ('actualClassifier', classifier)
        ])
        model = clf.fit(X_train, y_train)
        # Append to accuracy_model the accuracy of the model
        accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)

    # Print the accuracy    
    print(accuracy_model)

In [ ]:
classifiers = [LogisticRegression(),MultinomialNB(alpha=1),DecisionTreeClassifier(),RandomForestClassifier(),
                   SGDClassifier(loss='hinge', penalty='l2', alpha=0.001, random_state=42, max_iter=5, tol=None),
                   SGDClassifier(loss='log', penalty='l2', alpha=0.001, random_state=42, max_iter=5, tol=None),
                   KNeighborsClassifier(n_neighbors=7, weights = 'distance',p = 1)
                 ]

classifier_names = ['Logistic Regression','MultinomialNB','Decision-Tree Classifier',
                       'Random-Forest Classifier','SGDC - svc - Classifier','SGDC - lr - Classifier','K-Neighbors Classifier']
features = [feats_lr, feats_nb, feats, feats_rf, feats_sgd_svc, feats_sgd_lr, feats]
i = -1
for classifier in classifiers:
    i = i+1
    print(classifier_names[i])
    acc_cross_val(classifier, features[i], X)

## Multinomial Naive Bayes

In [67]:
model_nb = Pipeline([
    ('features', feats_nb),
    ('multinomialNB', MultinomialNB(alpha=1))
])

In [68]:
save_prediction_accuracy(model_nb,X_train,y_train,X_test,y_test)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [ ]:
model_nb.fit(X,y)

In [ ]:
filename = 'models/nb_model.sav'
pickle.dump(model_nb, open(filename, 'wb'))

## Logistic Regression

In [71]:
#model_lr = LogisticRegression()
model_lr = Pipeline([
    ('features', feats_lr),
    ('logisticReg', LogisticRegression())
])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
save_prediction_accuracy(model_lr,X_train,y_train,X_test,y_test)

Model Performance
Accuracy = 96.00%.


In [ ]:
model_lr.fit(X,y)

In [ ]:
filename = 'models/lr_model.sav'
pickle.dump(model_lr, open(filename, 'wb'))

## Decision Tree

In [ ]:
model_dt = Pipeline([
    ('features', feats),
    ('decisionTree', DecisionTreeClassifier())
])

In [ ]:
save_prediction_accuracy(model_dt,X_train,y_train,X_test,y_test)

In [ ]:
model_dt.fit(X,y)

In [ ]:
filename = 'models/dt_model.sav'
pickle.dump(model_dt, open(filename, 'wb'))

## KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3, weights = 'distance',p = 1)
model_knn = Pipeline([
    ('features', feats),
    ('KNN', knn)
])

In [ ]:
save_prediction_accuracy(model_knn,X_train,y_train,X_test,y_test)

In [ ]:
model_knn.fit(X,y)

In [ ]:
filename = 'models/knn_model.sav'
pickle.dump(model_knn, open(filename, 'wb'))

## MultiLayer Perceptron

In [ ]:
# muy lento
'''mlp = MLPClassifier()
model_mlp = Pipeline([
    ('features', feats),
    ('MLP', mlp)
])'''

In [ ]:
#save_prediction_accuracy(model_mlp,X_train,y_train,X_test,y_test)

In [ ]:
#model_mlp.fit(X,y)

In [ ]:
#filename = 'models/mlp_model.sav'
#pickle.dump(model_mlp, open(filename, 'wb'))

## SGD (Stochastic Gradient Descent)

In [ ]:
model_sgd_svc = Pipeline([
    ('features', feats_sgd_svc),
    ('sgd', SGDClassifier(loss='hinge', penalty='l2', alpha=0.001, random_state=42, max_iter=5, tol=None))
])

In [ ]:
save_prediction_accuracy(model_sgd_svc,X_train,y_train,X_test,y_test)

In [ ]:
model_sgd_svc.fit(X,y)

In [ ]:
filename = 'models/sgd_model_svc.sav'
pickle.dump(model_sgd_svc, open(filename, 'wb'))

In [ ]:
model_sgd_lr = Pipeline([
    ('features', feats_sgd_lr),
    ('sgd', SGDClassifier(loss='log', penalty='l2', alpha=0.001, random_state=42, max_iter=5, tol=None))
])

In [ ]:
save_prediction_accuracy(model_sgd_lr,X_train,y_train,X_test,y_test)

In [ ]:
model_sgd_lr.fit(X,y)

In [ ]:
filename = 'models/sgd_model_lr.sav'
pickle.dump(model_sgd_lr, open(filename, 'wb'))

# Ensambles

## Voting

In [ ]:
from sklearn.ensemble import VotingClassifier

# 1ro con todos los modelos
vote  = VotingClassifier(estimators=[('lr', model_lr), ('nb', model_nb), ('sgd_lr', model_sgd_lr), ('sgd_svc', model_sgd_svc)], 
                          voting='hard')

In [ ]:
save_prediction_accuracy(vote,X_train,y_train,X_test,y_test)

## Stacking

In [ ]:
sclf = StackingClassifier(classifiers=[model_nb,model_lr],
                          meta_classifier=LogisticRegression()) #,model_mlp

In [ ]:
save_prediction_accuracy(sclf,X_train,y_train,X_test,y_test)

## Boosting

### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(random_state = 1, n_estimators=100)

model_ada = Pipeline([
    ('features', feats),
    ('adaboost', ada)
])

In [ ]:
save_prediction_accuracy(model_ada,X_train,y_train,X_test,y_test)

### XGBoost

In [ ]:
model_xgb = Pipeline([
    ('features', feats),
    ('xgboost', XGBClassifier())
])

In [ ]:
save_prediction_accuracy(model_xgb,X_train,y_train,X_test,y_test)

### LightGBM

In [ ]:
model_lgbm = Pipeline([
    ('features', feats),
    ('lgbm', lgb.LGBMClassifier())
])

In [ ]:
save_prediction_accuracy(model_lgbm,X_train,y_train,X_test,y_test)

## Bagging

### Random Forest

In [ ]:
model_rf = Pipeline([
    ('features', feats),
    ('rf', RandomForestClassifier())
])

In [ ]:
save_prediction_accuracy(model_rf,X_train,y_train,X_test,y_test)

# Optimization

## Hyperparametros

## Funciones de optimizacion

In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization

In [47]:
def GridSearchCV_models(model_name, model, parameters, X_train, y_train, X_test, y_test, _cv):
    
    pipe_clf = Pipeline([
        ('features', feats),
        (model_name, model),
    ])
        
    best_clf = GridSearchCV(estimator = pipe_clf, param_grid = parameters, cv = _cv, verbose=2, n_jobs = -1)
    best_clf.fit(X_train, y_train)
        
    evaluate(best_clf, X_test, y_test)
            
    return best_clf.best_params_

In [48]:
def RandomizedSearchCV_models(model_name, model, parameters, X_train, y_train, X_test, y_test, _n_iter, _cv):
    
    pipe_clf = Pipeline([
        ('features', feats),
        (model_name, model),
    ])
        
    best_clf = RandomizedSearchCV(estimator = pipe_clf, param_distributions = parameters, n_iter = _n_iter, cv = _cv, verbose=2, random_state=42, n_jobs = -1)
    best_clf.fit(X_train, y_train)
        
    evaluate(best_clf, X_test, y_test)
            
    return best_clf.best_params_

## Algoritmos optimizados

### Logistic Regression

** RandomizedSearchCV **

In [49]:
model_name = "LR"
model = LogisticRegression()
parameters = {
    'LR__penalty':['l1', 'l2', 'elasticnet','none'],
    'LR__C': [0.1, 0.5, 0.25, 0.75, 1, 2, 3, 2.5, 1.5],
    'LR__solver':[ 'lbfgs', 'liblinear', 'sag', 'saga'],
    'LR__multi_class': ['auto', 'ovr', 'multinomial'],
    'LR__max_iter': np.arange(100,2000,200),
    "LR__n_jobs": [-1]
}
_n_iter = 10
_cv = 5

In [216]:
# RandomizedSearchCV

best_params_LR_random = RandomizedSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _n_iter, _cv)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.0min finished



MODEL
RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('features',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('cant_palabras',
                                                                              Pipeline(memory=None,
                                                                                       steps=[('selector',
                                                                                               NumberSelector(key='#palabras'))],
                                                                                       verbose=False)),
                                                                             ('links_cant',
                                                                              Pipeline(memory=None,
                                          

In [217]:
print(best_params_LR_random)

{'LR__solver': 'saga', 'LR__penalty': 'l2', 'LR__n_jobs': -1, 'LR__multi_class': 'multinomial', 'LR__max_iter': 1500, 'LR__C': 2}


** GridSearchCV_models **

In [ ]:
model_name = "LR"
model = LogisticRegression()
parameters = {
    'LR__penalty':['l2'],
    'LR__C': [0.1,0.5,0.25,0.75,1,2,5,10],
    'LR__solver':[ 'lbfgs', 'liblinear', 'sag', 'saga'],
    'LR__multi_class': ['multinomial'],
    'LR__max_iter': np.arange(100,1500,100),
    "LR__n_jobs": [-1]
}
_n_iter = 5
_cv = 3

In [ ]:
best_params_LR_grid = GridSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _cv)

In [ ]:
print(best_params_LR_grid)

** Bayesian Optimization **

In [162]:
LR_parameters = {
    'C': [0.01,10],
    'max_iter':(100,2000),
}

In [166]:
def LR_eval( C, max_iter):
  
    model = LogisticRegression(
        penalty = "l2",
        C = C,
        solver = "saga",
        multi_class = "multinomial",
        max_iter = max_iter,
    )               
    
    pipe_clf = Pipeline([
        ('features', feats),
        ("LR", model),
    ])
                                 
    pipe_clf.fit(X_train.fillna(0),y_train)
                   
    return (accuracy_score(y_test, pipe_clf.predict(X_test.fillna(0)))*100)

In [167]:
lrBO = BayesianOptimization(LR_eval, LR_parameters)

In [168]:
lrBO.maximize(init_points=2,n_iter=20)

|   iter    |  target   |     C     | max_iter  |
-------------------------------------------------
|  1        |  80.83    |  0.2668   |  1.56e+03 |
|  2        |  80.56    |  3.751    |  1.348e+0 |
|  3        |  80.76    |  1.874    |  1.558e+0 |
|  4        |  75.11    |  0.01     |  1.653e+0 |
|  5        |  80.43    |  6.0      |  1.38e+03 |
|  6        |  80.43    |  9.882    |  1.304e+0 |
|  7        |  75.05    |  0.01     |  1.25e+03 |
|  8        |  80.83    |  0.849    |  1.324e+0 |
|  9        |  80.24    |  10.0     |  1.579e+0 |
|  10       |  80.96    |  1.012    |  1.42e+03 |
|  11       |  80.3     |  8.42     |  1.451e+0 |
|  12       |  80.83    |  1.223    |  1.507e+0 |
|  13       |  80.96    |  1.677    |  194.6    |
|  14       |  76.1     |  0.01     |  158.3    |
|  15       |  80.96    |  1.632    |  213.6    |
|  16       |  80.83    |  0.2775   |  1.483e+0 |
|  17       |  80.37    |  9.667    |  1.402e+0 |
|  18       |  80.83    |  7.87     |  247.2    |


In [169]:
lrBO.max

{'params': {'C': 1.011702931147121, 'max_iter': 1419.6441643789612},
 'target': 80.9586342744583}

### SGDClassifier

**RandomizedSearchCV** 

In [311]:
model_name = "SGD"
model = SGDClassifier()
parameters = {
    'SGD__penalty':['l1', 'l2', 'elasticnet'],
    "SGD__loss":  ['hinge', 'log', 'modified_huber', 'perceptron'],
    'SGD__alpha': [0.1, 0.5, 0.25, 0.75, 1, 0.001,0.01,0.0001,0.15,0.2],
    'SGD__epsilon':[0.1, 0.5, 0.25, 0.75, 1, 0.001,0.01,0.0001,0.2,0.15],
    'SGD__learning_rate': ["optimal","constant","invscaling","adaptive"],
    'SGD__max_iter': np.arange(50,3000,50),
    "SGD__n_jobs": [-1],
    "SGD__eta0":[0.1,0.001,1,0.0001,0.00001,1]
    #"SGD__average": [True, False],
    #"SGD__early_stopping": [True, False],
    #"SGD__fit_intercept": [True, False],
    #"SGD__shuffle": [True, False]
}

_n_iter = 15
_cv = 15

In [312]:
# RandomizedSearchCV

best_params_SGD_random = RandomizedSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _n_iter, _cv)

Fitting 15 folds for each of 15 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:   49.2s finished



MODEL
RandomizedSearchCV(cv=15, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('features',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('cant_palabras',
                                                                              Pipeline(memory=None,
                                                                                       steps=[('selector',
                                                                                               NumberSelector(key='#palabras'))],
                                                                                       verbose=False)),
                                                                             ('links_cant',
                                                                              Pipeline(memory=None,
                                         

In [313]:
print(best_params_SGD_random)

{'SGD__penalty': 'elasticnet', 'SGD__n_jobs': -1, 'SGD__max_iter': 300, 'SGD__loss': 'log', 'SGD__learning_rate': 'optimal', 'SGD__eta0': 1, 'SGD__epsilon': 0.2, 'SGD__alpha': 0.001}


In [317]:
model_name = "SGD"
model = SGDClassifier()
parameters = {
    'SGD__penalty':[ 'elasticnet'],
    "SGD__loss":  [ 'modified_huber'],
    'SGD__alpha': [ 0.001, 0.005, 0.002,0.0008],
    'SGD__epsilon':[ 0.01, 0.05, 0.02,0.008],
    'SGD__learning_rate': ["adaptive"],
    'SGD__max_iter': np.arange(800,1500,100),
    "SGD__n_jobs": [-1],
    "SGD__eta0":[0.1,0.02,0.05,0.5,0.2],
    #"SGD__average": [True, False],
    #"SGD__early_stopping": [True, False],
    #"SGD__fit_intercept": [True, False],
    #"SGD__shuffle": [True, False]
}

_n_iter = 15
_cv = 15

In [318]:
best_params_SGD_random = RandomizedSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _n_iter, _cv)

Fitting 15 folds for each of 15 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.2min finished



MODEL
RandomizedSearchCV(cv=15, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('features',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('cant_palabras',
                                                                              Pipeline(memory=None,
                                                                                       steps=[('selector',
                                                                                               NumberSelector(key='#palabras'))],
                                                                                       verbose=False)),
                                                                             ('links_cant',
                                                                              Pipeline(memory=None,
                                         

In [319]:
print(best_params_SGD_random)

{'SGD__penalty': 'elasticnet', 'SGD__n_jobs': -1, 'SGD__max_iter': 1000, 'SGD__loss': 'modified_huber', 'SGD__learning_rate': 'adaptive', 'SGD__eta0': 0.05, 'SGD__epsilon': 0.01, 'SGD__alpha': 0.002}


**GridSearchCV_models**

In [320]:
model_name = "SGD"
model = SGDClassifier()
parameters = {
    'SGD__penalty':[ 'elasticnet'],
    "SGD__loss":  [ 'modified_huber'],
    'SGD__alpha': [ 0.001, 0.005, 0.002,0.0008],
    'SGD__epsilon':[ 0.01, 0.05, 0.02,0.008],
    'SGD__learning_rate': ["adaptive"],
    'SGD__max_iter': np.arange(800,1500,100),
    "SGD__n_jobs": [-1],
    "SGD__eta0":[0.1,0.02,0.05,0.5,0.2],
    "SGD__average": [True, False],
    "SGD__early_stopping": [True, False],
    "SGD__fit_intercept": [True, False],
    "SGD__shuffle": [True, False]
}

In [ ]:
best_params_SGD_grid = GridSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, 3)

Fitting 3 folds for each of 8960 candidates, totalling 26880 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 13.0min


In [ ]:
print(best_params_SGD_random)

In [ ]:
model_name = "SGD"
model = SGDClassifier()
parameters = {
    'SGD__penalty':[ 'elasticnet'],
    "SGD__loss":  [ 'modified_huber'],
    'SGD__alpha': [ 0.001, 0.005, 0.002,0.0008],
    'SGD__epsilon':[ 0.01, 0.05, 0.02,0.008],
    'SGD__learning_rate': ["adaptive"],
    'SGD__max_iter': np.arange(800,1500,100),
    "SGD__n_jobs": [-1],
    "SGD__eta0":[0.1,0.02,0.05,0.5,0.2],
}

In [295]:
best_params_SGD_grid = GridSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, 3)

Fitting 3 folds for each of 448 candidates, totalling 1344 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1344 out of 1344 | elapsed:  5.0min finished



MODEL
GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('cant_palabras',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('selector',
                                                                                         NumberSelector(key='#palabras'))],
                                                                                 verbose=False)),
                                                                       ('links_cant',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('selector',
       

In [298]:
print(best_params_SGD_random)

{'SGD__penalty': 'elasticnet', 'SGD__n_jobs': -1, 'SGD__max_iter': 1000, 'SGD__loss': 'modified_huber', 'SGD__learning_rate': 'adaptive', 'SGD__eta0': 0.05, 'SGD__epsilon': 0.02, 'SGD__alpha': 0.002}


In [ ]:
best = {'SGD__penalty': 'elasticnet', 'SGD__n_jobs': -1, 'SGD__max_iter': 1000, 'SGD__loss': 'modified_huber', 'SGD__learning_rate': 'adaptive', 'SGD__eta0': 0.05, 'SGD__epsilon': 0.02, 'SGD__alpha': 0.002}

** Bayesian Optimization **

In [170]:
SGD_parameters = {
    'epsilon': (0.0001,0.5),
    'alpha': (0.0001,0.5),
    "eta0": (0.001,0.5),
    "max_iter": (100,1500)
}

In [172]:
def SGD_eval(alpha, epsilon, eta0, max_iter):
  
    model = SGDClassifier(
        
    penalty = 'elasticnet',
    loss ='modified_huber',
    learning_rate= "adaptive",
    n_jobs=-1,
    
    alpha= alpha,
    epsilon = epsilon,
    max_iter= max_iter,
    eta0=eta0
    )               
    
    pipe_clf = Pipeline([
        ('features', feats),
        ("clf", model),
    ])
                                 
    pipe_clf.fit(X_train.fillna(0),y_train)
                   
    return (accuracy_score(y_test, pipe_clf.predict(X_test.fillna(0)))*100)

In [173]:
SGD_BO = BayesianOptimization(SGD_eval, SGD_parameters)

In [174]:
print(SGD_BO.space.keys)

['alpha', 'epsilon', 'eta0', 'max_iter']


In [175]:
SGD_BO.maximize()

|   iter    |  target   |   alpha   |  epsilon  |   eta0    | max_iter  |
-------------------------------------------------------------------------
|  1        |  57.45    |  0.1187   |  0.3183   |  0.02295  |  1.344e+0 |
|  2        |  56.27    |  0.0641   |  0.3518   |  0.4925   |  574.1    |
|  3        |  55.88    |  0.292    |  0.2749   |  0.3572   |  303.8    |
|  4        |  54.63    |  0.4997   |  0.4398   |  0.3856   |  1.211e+0 |
|  5        |  55.88    |  0.3417   |  0.106    |  0.2487   |  1.38e+03 |
|  6        |  54.83    |  0.4358   |  0.02889  |  0.1193   |  1.343e+0 |
|  7        |  55.02    |  0.2615   |  0.3839   |  0.4381   |  574.1    |
|  8        |  55.61    |  0.2764   |  0.08139  |  0.367    |  846.8    |
|  9        |  56.27    |  0.1837   |  0.4882   |  0.3148   |  1.429e+0 |
|  10       |  55.22    |  0.3945   |  0.05175  |  0.4156   |  458.2    |
|  11       |  55.42    |  0.3415   |  0.1945   |  0.1777   |  1.135e+0 |
|  12       |  80.83    |  0.002686 | 

In [176]:
SGD_BO.max

{'params': {'alpha': 0.002686097902550382,
  'epsilon': 0.2776848693533973,
  'eta0': 0.08273258247904128,
  'max_iter': 388.5705756508965},
 'target': 80.82731451083389}

### Multinomial

** RandomizedSearchCV **

In [ ]:
model_name = "MNB"
model = MultinomialNB()
parameters = {
    'MNB__alpha':[3,2,1, 0.75, 0.8, 0.6, 0.7,0.5, 1e-1, 1e-2,1e-8,10,20,50],
    'MNB__fit_prior': [True,False],
}
_n_iter = 20
_cv = 10

In [130]:
# RandomizedSearchCV

best_params_LR_random = RandomizedSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _n_iter, _cv)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   43.3s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   54.2s finished



MODEL
RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('features',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('cant_palabras',
                                                                              Pipeline(memory=None,
                                                                                       steps=[('selector',
                                                                                               NumberSelector(key='#palabras'))],
                                                                                       verbose=False)),
                                                                             ('links_cant',
                                                                              Pipeline(memory=None,
                                         

In [131]:
print(best_params_LR_random)

{'MNB__fit_prior': False, 'MNB__alpha': 1}


** GridSearchCV **

In [132]:
# GridSearchCV_models

best_params_LR_grid = GridSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _cv)

Fitting 10 folds for each of 28 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   38.0s
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:  1.2min finished



MODEL
GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('cant_palabras',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('selector',
                                                                                         NumberSelector(key='#palabras'))],
                                                                                 verbose=False)),
                                                                       ('links_cant',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('selector',
      

In [133]:
print(best_params_LR_grid)

{'MNB__alpha': 1, 'MNB__fit_prior': False}


** Bayesian Optimization **

In [142]:
MNB_parameters = {
    'alpha':(1e-2, 5),
}

In [143]:
def MNB_eval(alpha):
  
    model = MultinomialNB(
        alpha = alpha,
        fit_prior = False
    )               
    
    pipe_clf = Pipeline([
        ('features', feats),
        ("clf", model),
    ])
                                 
    pipe_clf.fit(X_train.fillna(0),y_train)
                   
    return (accuracy_score(y_test, pipe_clf.predict(X_test.fillna(0)))*100)

In [144]:
MNB_BO = BayesianOptimization(MNB_eval, MNB_parameters)

In [145]:
print(MNB_BO.space.keys)

['alpha']


In [146]:
MNB_BO.maximize()

|   iter    |  target   |   alpha   |
-------------------------------------
|  1        |  78.33    |  3.144    |
|  2        |  79.84    |  1.795    |
|  3        |  80.24    |  1.332    |
|  4        |  77.48    |  4.809    |
|  5        |  78.73    |  2.903    |
|  6        |  77.87    |  0.01     |
|  7        |  80.04    |  0.9428   |
|  8        |  80.17    |  1.175    |
|  9        |  79.97    |  1.494    |
|  10       |  79.32    |  2.275    |
|  11       |  77.81    |  3.958    |
|  12       |  80.04    |  0.9428   |
|  13       |  80.24    |  0.6351   |
|  14       |  80.24    |  0.7443   |
|  15       |  80.11    |  1.269    |
|  16       |  79.51    |  2.014    |
|  17       |  79.84    |  0.4175   |
|  18       |  78.92    |  2.57     |
|  19       |  80.11    |  1.069    |
|  20       |  77.94    |  3.541    |
|  21       |  80.11    |  0.5413   |
|  22       |  80.24    |  1.395    |
|  23       |  80.04    |  0.6917   |
|  24       |  77.54    |  4.369    |
|  25       

In [152]:
MNB_BO.max

{'params': {'alpha': 0.7931196386847666}, 'target': 80.3676953381484}

### RandomForestClassifier

** RandomizedSearchCV **

In [ ]:
model_name = "RF"
model = RandomForestClassifier()

parameters = {
    "RF__criterion": ["gini", "entropy"],
    'RF__n_estimators': np.arange(100,500,100),
    'RF__max_features': ['auto', "sqrt", "log2"],
    #'RF__max_depth': [np.arange(1, 500, 10)],
    #'RF__min_samples_split': np.arange(0, 5, 1),
    #'RF__min_samples_leaf': np.arange(0, 5, 1),
    #"RF__min_impurity_decrease": [0.000001, 0.0001,0.0000001, 0.001, 0.01],
    "RF__n_jobs": [-1]
    #"RF__min_weight_fraction_leaf": [0.000001, 0.0001,0.0000001, 0.001, 0.01]
}
_n_iter = 10
_cv = 3

In [ ]:
best_params_RF_random = RandomizedSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _n_iter, _cv)

In [184]:
model_name = "RF"
model = RandomForestClassifier()

parameters = {
    "RF__criterion": ["gini", "entropy"],
    'RF__n_estimators': np.arange(100,500,100),
    'RF__max_features': ['auto', "sqrt", "log2"],
    #'RF__max_depth': [np.arange(1, 500, 10)],
    #'RF__min_samples_split': np.arange(0, 5, 1),
    #'RF__min_samples_leaf': np.arange(0, 5, 1),
    "RF__ccp_alpha":[ 0.0001], # Pruning
    #"RF__min_impurity_decrease": [0.000001, 0.0001,0.0000001, 0.001, 0.01],
    "RF__n_jobs": [-1]
    #"RF__min_weight_fraction_leaf": [0.000001, 0.0001,0.0000001, 0.001, 0.01]
}
_n_iter = 10
_cv = 3

In [185]:
best_params_RF_random = RandomizedSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _n_iter, _cv)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
print(best_params_RF_random)

** GridSearchCV **

In [ ]:
best_params_LR_grid = GridSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _cv)

** Bayesian Optimization **

In [ ]:
def rf_eval(n_estimators, min_samples_split, max_features):
    
    rfc = RandomForestClassifier(n_estimators=int(n_estimators),
                               min_samples_split=int(min_samples_split),
                               max_features=min(max_features, 0.999),
                               random_state=2,
                               n_jobs=-1)
    rf= Pipeline([
        ('features', feats),
        ('randomforest', rfc )
        ])
    rf.fit(X_train.fillna(0),y_train)
    return (accuracy_score(y_test,rf.predict(X_test.fillna(0)))*100)

In [ ]:
rfBO = BayesianOptimization(rf_eval, {
    'n_estimators':(100,500),
    'min_samples_split':(2,10),
    'max_features':(1,100)
})

In [ ]:
rfBO.maximize(init_points=1, n_iter=5)

In [ ]:
rfBO.max

###  LGBMClassifier

In [191]:
from lightgbm import LGBMClassifier

model_name = "LGBM"
model = LGBMClassifier()

parameters ={
    'LGBM__objective': ['binary'],
    'LGBM__num_leaves': np.arange(25,70,4),
    'LGBM__learning_rate':[0.005,0.01,0.05,0.1,0.3],
    'LGBM__n_estimators': np.arange(25,200,15),
    'LGBM__max_depth': np.arange(5,13,1),
    'LGBM__min_split_gain': [0.001,0.01,0.1,0.2],
    'LGBM__bagging_fraction': np.arange(0.8,1.01,0.1),
    'LGBM__feature_fraction': np.arange(0.1,0.91,0.2)
}

_n_iter = 10
_cv = 10

In [192]:
best_params_LGBM_random = RandomizedSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _n_iter, _cv)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   35.8s finished



MODEL
RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('features',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('cant_palabras',
                                                                              Pipeline(memory=None,
                                                                                       steps=[('selector',
                                                                                               NumberSelector(key='#palabras'))],
                                                                                       verbose=False)),
                                                                             ('links_cant',
                                                                              Pipeline(memory=None,
                                         

### GradientBoostingClassifier

In [196]:
from sklearn.ensemble import GradientBoostingClassifier

model_name = "GBC"
model = GradientBoostingClassifier()

parameters = {
    'GBC__max_features':['auto', 'sqrt', 'log2'],
                             'GBC__max_leaf_nodes': [None,1,2,3,4,5,6,8],
                             'GBC__min_weight_fraction_leaf': np.linspace(0.0, 0.5, 1,endpoint=True),
                             'GBC__learning_rate': np.arange(0.1, 0.5, 0.05),
                             'GBC__min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True),
                             'GBC__min_samples_leaf': np.arange(0.1, 0.5, 5),  
                             'GBC__max_features' : list(range(1,100)),
                             'GBC__max_depth': [n for n in range(0,50,5)],
                             'GBC__n_estimators': [n for n in range(0,1000,10)],
                             'GBC__subsample': np.arange(0.3, 1,0.1),
                             'GBC__loss': ['deviance'],
                             'GBC__warm_start': [True, False],
                             'GBC__presort': ['auto'],

}

In [197]:
best_params_GBC_random = RandomizedSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _n_iter, _cv)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.2min finished



MODEL
RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('features',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('cant_palabras',
                                                                              Pipeline(memory=None,
                                                                                       steps=[('selector',
                                                                                               NumberSelector(key='#palabras'))],
                                                                                       verbose=False)),
                                                                             ('links_cant',
                                                                              Pipeline(memory=None,
                                         

### DesicionTreeClassifier

In [199]:

model_name = "DT"
model = DecisionTreeClassifier()

parameters = {
    "DT__criterion": ["entropy","gini"],
    'DT__max_features': ['auto', 'sqrt','log2'],
    'DT__max_depth': np.linspace(1, 500, 10, endpoint=True),
    'DT__min_samples_split': np.linspace(0.0, 1.0, 10, endpoint=True),
    'DT__min_samples_leaf': np.arange(0.0, 1,0.05),
}

In [200]:
best_params_GBC_random = RandomizedSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _n_iter, _cv)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   17.0s finished



MODEL
RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('features',
                                              FeatureUnion(n_jobs=None,
                                                           transformer_list=[('cant_palabras',
                                                                              Pipeline(memory=None,
                                                                                       steps=[('selector',
                                                                                               NumberSelector(key='#palabras'))],
                                                                                       verbose=False)),
                                                                             ('links_cant',
                                                                              Pipeline(memory=None,
                                         

### XGBClassifier

In [ ]:
model_name = "XGB"
model = XGBClassifier()

parameters ={
    "XGB__criterion": ["entropy","gini"],
    'XGB__n_estimators': np.arange(10,2000,200),
    'XGB__max_features': ['auto', 'sqrt','log2'],
    'XGB__max_depth': np.arange(1, 500, 10),
    'XGB__min_samples_split': np.linspace(0.0, 1.0, 10, endpoint=True),
    'XGB__min_samples_leaf': np.arange(1, 10, 1),
    'XGB__objective': ['binary:logistic'],
    'XGB__learning_rate':np.arange(0.1,0.5,0.1),
    'XGB__gamma':np.arange(0,0.5,0.1),
    'XGB__subsample':np.arange(0.6,1,0.1),
    'XGB__reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
    'XGB__tree_method':['auto', 'exact', 'approx', 'hist', 'gpu_hist']
}

_n_iter = 11
_cv = 3

In [ ]:
best_params_XGB_random = RandomizedSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _n_iter, _cv)

### MLPClassifier

In [207]:
model_name = "MLPC"
model = MLPClassifier()

parameters = {'MLPC__hidden_layer_sizes':[(n,n,n) for n in range(1,30)],
                'MLPC__activation':['identity', 'logistic', 'tanh', 'relu'],
                              'MLPC__alpha':[1e-06,1e-05,1e-04,1e-03,1e-02,1e-01,1],
                              'MLPC__beta_1':np.arange(0.0,1,0.01),
                              'MLPC__beta_2':np.arange(0.0,1,0.01),
                              'MLPC__early_stopping':[True, False],
                              'MLPC__epsilon':[1e-07,1e-08,1e-06, 1e-09, 1e-010, 1e-11],
                              'MLPC__learning_rate':['constant', 'adaptive'],
                              'MLPC__solver':['adam', 'lbfgs', 'sgd'],
                              'MLPC__validation_fraction':np.arange(0.15,0.5,0.01),
                               'MLPC__max_iter':[200,300,400],
                             }

In [ ]:
best_params_MLPC_random = RandomizedSearchCV_models(model_name, model, parameters, X_train.fillna(0), y_train, X_test.fillna(0), y_test, _n_iter, _cv)

# PREDICT ON TEST AND SAVE

In [ ]:
# replace "model" with the final model name
model.fit(X,y)

In [ ]:
# replace "model" with the final model name and name the file (dont forget the timestamp!)
save_prediction(model, test.text, 'some_name.csv')

# Vectorizacion del texto con Word2vec

In [82]:
import gensim
url = 'GoogleNews-vectors-negative300.bin.gz'
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url,binary=True)
data = pd.DataFrame()
for frase in train.text:
    temp = pd.DataFrame()
    for word in frase.split(' '):
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index=True)
        except:
            pass
    mean = temp.mean()
    data = data.append(mean, ignore_index=True)
data.shape

(7613, 300)

In [83]:
pd.isnull(data).sum().sum()

5700

In [84]:
data['target'] = train['target']
data = data.dropna()

In [86]:
train_X, test_X, train_y, test_y = train_test_split(data.drop('target',axis=1), data['target'], test_size=0.2, random_state=1)
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((6075, 300), (1519, 300), (6075,), (1519,))

# Modelos

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier()

In [ ]:
save_prediction_accuracy 

In [1]:
from sklearn.ensemble import AdaBoostClassifier

model=AdaBoostClassifier(n_estimators=800, random_state=1)
model.fit(train_X,train_y)

predictions = model.predict(test_X)
accuracy = round(accuracy_score(test_y,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

KeyboardInterrupt: 

In [92]:
model = LogisticRegression()
model.fit(train_X,train_y)

predictions = model.predict(test_X)
accuracy = round(accuracy_score(test_y,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

Model Performance
Accuracy = 78.00%.


In [93]:
model = RandomForestClassifier()
model.fit(train_X,train_y)

predictions = model.predict(test_X)
accuracy = round(accuracy_score(test_y,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

Model Performance
Accuracy = 78.00%.
